In [1]:
"""
Code cell #1
"""

"""
Functions for hierarchical clustering that consider the symmetries of the system.
"""

import pickle
import numpy as np
import matplotlib.pyplot as plt

from numba import jit
from tqdm import tqdm

from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [2]:
"""
Code cell #2
"""


def substrate_exchange(param):
    """
    Exchange parameters between substrate a and b.
    
    Parametres
    --------------
    param : array, shape (32)
    [ka1, ..., kb1, ..., Kma1, ..., Kmb1, ...]
    
    Returns
    ----------
    param : array, shape (32) 
    [kb1, ..., ka1, ..., Kmb1, ..., Kma1, ...]
    """
    
    param = param[[8, 9, 10, 11, 12, 13, 14, 15, 0, 1, 2, 3, 4, 5, 6, 7,
                                24, 25, 26, 27, 28, 29, 30, 31, 16, 17, 18, 19, 20, 21, 22, 23]]
    return(param)

def site_exchange(param):
    """
    Do site-exchange on both substrate a and b.
    
    Parametres
    --------------
    param : array, shape (32)
    [ka1, ..., kb1, ..., Kma1, ..., Kmb1, ...]
    
    Returns
    ----------
    param : array, shape (32) 
    [ka2, ..., kb2, ..., Kma2, ..., Kmb2, ...]
    """
    
    param = param[[1, 0, 3, 2, 5, 4, 7, 6, 9, 8, 11, 10, 13, 12, 15, 14, 
                               17, 16, 19, 18, 21, 20, 23, 22, 25, 24, 27, 26, 29, 28, 31, 30]]
    return(param)

def site_exchange_substrate_a(param):
    """
    Do site-exchange on both substrate a only.
    
    Parametres
    --------------
    param : array, shape (32)
    [ka1, ..., kb1, ..., Kma1, ..., Kmb1, ...]
    
    Returns
    ----------
    param : array, shape (32) 
    [ka2, ..., kb1, ..., Kma2, ..., Kmb1, ...]
    """
    
    param = param[[1, 0, 3, 2, 5, 4, 7, 6, 8, 9, 10, 11, 12, 13, 14, 15,
                               17, 16, 19, 18, 21, 20, 23, 22, 24, 25, 26, 27, 28, 29, 30, 31]]
    return(param)

def site_exchange_substrate_b(param):
    """
    Do site-exchange on both substrate b only.
    
    Parametres
    --------------
    param : array, shape (32)
    [ka1, ..., kb1, ..., Kma1, ..., Kmb1, ...]
    
    Returns
    ----------
    param : array, shape (32) 
    [ka1, ..., kb2, ..., Kma1, ..., Kmb2, ...]
    """
    
    param = param[[0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 11, 10, 13, 12, 15, 14,
                                16, 17, 18, 19, 20, 21, 22, 23, 25, 24, 27, 26, 29, 28, 31, 30]]
    return(param)

def kinase_exchange(param):
    """
    Do kinase-exchange.
    
    Parametres
    --------------
    param : array, shape (32)
    [ka1, ..., kb1, ..., Kma1, ..., Kmb1, ...]
    
    Returns
    ----------
    param : array, shape (32) 
    [ka7, ..., kb7, ..., Kma7, ..., Kmb7, ...]
    """
        
    param = param[[6, 7, 4, 5, 2, 3, 0, 1, 14, 15, 12, 13, 10, 11, 8, 9,
                                22, 23, 20, 21, 18, 19, 16, 17, 30, 31, 28, 29, 26, 27, 24, 25]]
    return(param)

def sym_copy(param):
    """
    Make every possible symmetric parameter sets for a given paramter set.
    
    Paramters
    -------------
    param : array, shape (32)
    A paramter set.
    
    Returns
    ----------
    sym_params : array, shape (16, 32)
    Symmetric parameter sets.
    """
    
    sym_params = np.empty((16, param.shape[0]))
    
    sym_params[0] = param
    
    # substrate sym copy
    sym_params[1] = substrate_exchange(param)
    
    # site-a sym copy
    sym_params[2] = site_exchange_substrate_a(sym_params[0])
    sym_params[3] = site_exchange_substrate_a(sym_params[1])
        
    # site-b sym copy
    sym_params[4] = site_exchange_substrate_b(sym_params[0])
    sym_params[5] = site_exchange_substrate_b(sym_params[1])
    sym_params[6] = site_exchange_substrate_b(sym_params[2])
    sym_params[7] = site_exchange_substrate_b(sym_params[3])
    
    # kinase sym copy
    sym_params[8] = kinase_exchange(sym_params[0])
    sym_params[9] = kinase_exchange(sym_params[1])
    sym_params[10] = kinase_exchange(sym_params[2])
    sym_params[11] = kinase_exchange(sym_params[3])
    sym_params[12] = kinase_exchange(sym_params[4])
    sym_params[13] = kinase_exchange(sym_params[5])
    sym_params[14] = kinase_exchange(sym_params[6])
    sym_params[15] = kinase_exchange(sym_params[7])

    return(sym_params)
       
def align_params_in_cluster(i_ref, cluster):
    """
    Align parameter sets in a cluster by considering the symmetries.
    
    Parameters
    --------------
    i_ref: int
    An index of a reference parameter set within the cluster.
    cluster: array
    A parameter set cluster.
    
    Returns
    ----------
    Aligned: array
    An aligned cluster.
    """
    
    reference = cluster[i_ref]
        
    for i, c in enumerate(cluster):
        idx = measure_paramset_distance(reference, c, return_index=True)
            
        if i==0:
            aligned = sym_copy(c)[idx]
        else:
            aligned = np.vstack((aligned, sym_copy(c)[idx]))
    return(aligned)
        
def measure_paramset_distance(ref, param, return_index=False):
    """
    Measure the euclidian distance between a reference parameter set and a given parameter set considering the symmetries.
    
    Parameters
    --------------
    ref : array, shape (32)
    A reference parameter set.
    param : array, shape (32)
    A parameter set.
    return_index : bool
    A flag to return which symmetric copy is the closest to the reference.
    """
    
    param_syms = sym_copy(param)
    ds = np.sum((param_syms - ref)**2, axis=1)
    if return_index==False:
        return(np.min(ds, axis=0))
    else:
        return(np.argmin(ds, axis=0))
        
def measure_cluster_distance(cl1, cl2, merge=False):
    """
    Measure the distance between given clusters.
    
    Paramters
    -------------
    cl1 : (list, scalar)
    The list contains a Indice array for the parameter sets that belong to cluster 1 within the all parameters.
    The sclar is the sum of the squared euclidian distance between the parameter sets and c.o.g..
    
    cl1 : (list, scalar)
    The list contains a Indice array for the parameter sets that belong to cluster 2 within the all parameters.
    The sclar is the sum of the squared euclidian distance between the parameter sets and c.o.g..
    
    merge : bool
    A flag to merge cluster 1 and 2.
    
    Returns
    ----------
    d : scalar
    The difference of L between before and after the merge.
    """
    
    c1, L1 = cl1[1], cl1[2]
    c2, L2 = cl2[1], cl2[2]
    
    cc = c1 + c2
    dmat = np.zeros((len(cc), len(cc)))
    for row in range(len(dmat)):
        for col in range(row+1, len(dmat)):
            tempd = measure_paramset_distance(cc[row], cc[col])
            dmat[row, col] = tempd
            dmat[col, row] = tempd
    i_ref = np.argmin(np.mean(dmat, axis=1))
    merged_c = list(align_params_in_cluster(i_ref, cc))
    merged_com = np.mean(merged_c, axis=0)
    
    merged_L = np.sum([measure_paramset_distance(c, merged_com) for c in c1]) + np.sum([measure_paramset_distance(c, merged_com) for c in c2])
    d = merged_L - L1 - L2
    if merge==False:
        return(d)
    else:
        return([merged_c, merged_L])
    

def return_linkage(data, idx, distance_matrix='init', Z='init'):
    """
    Find the next pair of clusters to merge.
    
    Paramters
    -------------
    data : [[scalar, list, scalar], [scalar, list, scalar], ...]
    [[idx_cluster, members, L], [.,.,.], ..., [.,.,.]]
    idx : scalar
    The index of the cluster.
    distance_matrix : matrix
    The distance matrix between clusters.
    Z : matrix
    Scipy compatitive linkage matrix. each col represents merged cluster1.
    
    Returns
    ----------
    data : [[scalar, list, scalar], [scalar, list, scalar], ...]
    The updated [[idx_cluster, members, L], [.,.,.], ..., [.,.,.]]
    idx : scalar
    The updated index of the cluster.
    distance_matrix : matrix
    The pdated distance matrix between clusters.
    Z : matrix
    The updated scipy compatitive linkage matrix. each col represents merged cluster1.
    """
    
    n_cluster = len(data)
    
    if distance_matrix=='init':
        distance_matrix = np.ones((n_cluster, n_cluster))*np.inf
        for row in tqdm(range(distance_matrix.shape[1])):
            for col in range(row+1, distance_matrix.shape[0]): # col > row
                distance_matrix[row, col] = measure_cluster_distance(data[row], data[col])
    else:
        for col in range(1, distance_matrix.shape[0]): # col > row
            distance_matrix[0, col] = measure_cluster_distance(data[0], data[col])
    
    i_closest  = np.argmin(np.ravel(distance_matrix))
    
    merge_1 = i_closest//n_cluster # merge_1 < merge_2
    merge_2  = i_closest%n_cluster
    
    merged_c, merged_L = measure_cluster_distance(data[merge_1], data[merge_2], merge=True)
    new_cluster = [idx, merged_c, merged_L]
    
    Z_new = np.asarray([data[merge_1][0], data[merge_2][0], merged_L, len(new_cluster[1])])
    del(data[merge_2])
    del(data[merge_1])
    data = [new_cluster] + data
    
    if Z=='init':
        Z = Z_new
    else:
        Z = np.vstack((Z, Z_new))
    
    new_d = np.ones((n_cluster-1, n_cluster-1))*np.inf
    new_d[1:merge_1+1, 1:merge_1+1]                            = distance_matrix[:merge_1, :merge_1]
    new_d[1:merge_1+1, merge_1+1:merge_2]              = distance_matrix[:merge_1, merge_1+1:merge_2]
    new_d[1:merge_1+1, merge_2:]                                   = distance_matrix[:merge_1, merge_2+1:]
    new_d[merge_1+1:merge_2, merge_1+1:merge_2] = distance_matrix[merge_1+1:merge_2, merge_1+1:merge_2]
    new_d[merge_1+1:merge_2, merge_2:]                     = distance_matrix[merge_1+1:merge_2, merge_2+1:]
    new_d[merge_2:, merge_2:]                                          = distance_matrix[merge_2+1:, merge_2+1:]
    
    distance_matrix = new_d
    
    return(data, idx+1, distance_matrix, Z)

def sym_ward_clustering(params):
    """
    Do ward clustering considering the symmetries of the system.
    
    Paramters
    -------------
    params : matrix
    Stacked parameters.
    
    Returns
    ----------
    Z : matrix
    Scipy compatitive linkage matrix
    dm : matrix
    Distance matrix between clusters.
    """
    
    data = [[idx, [c], 0] for idx, c in enumerate(params)]
    data, idx, dm, Z = return_linkage(data=data, idx=len(data))
    data, idx, dm, Z = return_linkage(data=data, idx=idx, distance_matrix=dm, Z=Z)
    
    i_iter = 1
    while Z[-1, -1]<params.shape[0]:
        data, idx, dm, Z = return_linkage(data=data, idx=idx, distance_matrix=dm, Z=Z)
        if i_iter%100==0:
            print(i_iter)
        i_iter += 1
        
    return(Z, dm)

In [3]:
"""
Code cell #3
"""

# params: chaos parameters you have confirmed.
with open('./chaosparam.pickle', 'rb') as f:
    params_for_clustering = pickle.load(f)

Z, D = sym_ward_clustering(params_for_clustering)

In [35]:
"""
Code cell #4
"""

def draw_dendrogram(*args, **kwargs):
    """
    https://joernhees.de/blog/2015/08/26/scipy-hierarchical-clustering-and-dendrogram-tutorial/
    """
    max_d = kwargs.pop('max_d', None)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    annotate_above = kwargs.pop('annotate_above', 0)

    ddata = dendrogram(*args, **kwargs)

    if not kwargs.get('no_plot', False):
        plt.title('Hierarchical Clustering Dendrogram (truncated)')
        plt.xlabel('sample index or (cluster size)')
        plt.ylabel('distance')
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            if y > annotate_above:
                plt.plot(x, y, 'o', c=c)
                plt.annotate('%.3g' % y, (x, y), xytext=(0, -5),
                             textcoords='offset points',
                             va='top', ha='center')
    return(ddata)

with open('./Z.pickle', 'rb') as f:
    Z = pickle.load(f)

draw_dendrogram(Z,
           truncate_mode='lastp',  # show only the last p merged clusters
           p=33, # show only the last p merged clusters
           leaf_rotation=90.,  # rotates the x axis labels
           leaf_font_size=0.,  # font size for the x axis labels
           show_contracted=False,  # to get a distribution impression in truncated
           annotate_above=200000,  # useful in small plots so annotations don't overlap
           max_d=40000,  # plot a horizontal cut-off line
          )
plt.title('Chaotic parameter set clusters')
plt.xlabel('Clusters')
plt.ylabel('Distance')
plt.yticks([0, 60000, 120000, 180000])
plt.hlines(max_d, xmin=0, xmax=1000, color='black')
plt.show()
plt.close()

In [51]:
# Which cluster each parameter set is classified into
i_clusters = fcluster(Z, max_d, criterion='distance') -1